# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [1]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
def build_classifiers():
    linear_regression = LinearRegression()
    linear_regression.fit(data_set, labels)
    
    knn = KNeighborsClassifier()
    knn.fit(data_set, labels)
    
    svc = SVC()
    svc.fit(data_set, labels)
    
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(data_set, labels)
    
    naive_bayes = GaussianNB()
    naive_bayes.fit(data_set, labels)
    
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(data_set, labels)
    
    return [linear_regression, knn, svc, decision_tree, naive_bayes, qda]

In [4]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier() # set here
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [5]:
from itertools import permutations
from operator import itemgetter
import pandas as pd

classifiers = build_classifiers()

results = []

for permutation in permutations(range(len(classifiers)), r=3):
    selected_classifiers = itemgetter(*permutation)(classifiers)
    predicted = build_stacked_classifier(selected_classifiers)
    accuracy = accuracy_score(test_labels, predicted)
    classifier_names = [type(classifier).__name__ for classifier in selected_classifiers]
    results.append((accuracy, *classifier_names))

results.sort(key=itemgetter(0), reverse=True)

column_names = ['Accuracy', 'Classifier 1', 'Classifier 2', 'Classifier 3']
data_for_df = [dict(zip(column_names, result)) for result in results[:12]]

df = pd.DataFrame(data_for_df)
df.index = df.index + 1

print(df)

    Accuracy                   Classifier 1                   Classifier 2  \
1       0.95         DecisionTreeClassifier           KNeighborsClassifier   
2       0.90           KNeighborsClassifier         DecisionTreeClassifier   
3       0.90           KNeighborsClassifier  QuadraticDiscriminantAnalysis   
4       0.90                     GaussianNB           KNeighborsClassifier   
5       0.90                     GaussianNB           KNeighborsClassifier   
6       0.90  QuadraticDiscriminantAnalysis           KNeighborsClassifier   
7       0.85           KNeighborsClassifier                            SVC   
8       0.85           KNeighborsClassifier                            SVC   
9       0.85           KNeighborsClassifier                            SVC   
10      0.85           KNeighborsClassifier                     GaussianNB   
11      0.85           KNeighborsClassifier                     GaussianNB   
12      0.85                            SVC           KNeighbors

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [7]:
def set_new_weights(model):
    predicted = model.predict(test_set)
    I = np.array([int(p != t) for p, t in zip(predicted, test_labels)])
    Z = np.sum(I)
    new_weights = (1 + I) / (1.0 * (1 + Z))
    return new_weights

Train the classifier with the code below:

In [8]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00373832 0.00373832 0.00373832 0.00186916 0.00373832 0.00186916
 0.00373832 0.00373832 0.00373832 0.00373832 0.00186916 0.00373832
 0.00373832 0.00373832 0.00186916 0.00186916 0.00373832 0.00186916
 0.00186916 0.00373832 0.00186916 0.00186916 0.00373832 0.00186916
 0.00186916 0.00186916 0.00186916 0.00186916 0.00373832 0.00186916
 0.00373832 0.00186916 0.00373832 0.00186916 0.00373832 0.00186916
 0.00373832 0.00186916 0.00373832 0.00186916 0.00373832 0.00186916
 0.00373832 0.00186916 0.00373832 0.00373832 0.00373832 0.00186916
 0.00186916 0.00373832 0.00373832 0.00186916 0.00186916 0.00373832
 0.00373832 0.00373832 0.00373832 0.00186916 0.00373832 0.00373832
 0.00186916 0.00186916 0.00373832 0.00373832 0.00186916 0.00373832
 0.00186916 0.00373832 0.00373832 0.00373832 0.00186916 0.00186916
 0.00373832 0.00186916 0.00373832 0.00186916 0.00373832 0.00186916
 0.00373832 0.00373832 0.00186916 0.00373832 0.00186916 0.00186916
 0.00186916 0.00373832 0.00186916 0.00373832 0.00373832 0.0018

Set the validation data set:

In [9]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [10]:
def get_prediction(x):
    predictions = np.array([classifier.predict(x) for classifier in classifiers])
    final_prediction = np.round(np.mean(predictions, axis=0)).astype(int)
    return final_prediction

Test it:

In [11]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
